In [1]:
import csv
import itertools
import numpy
import random
from CWHashingFamily import CWHashingFamily

SHINGLES_LEN = 4
PRIME = 997
BINS = 900

def shingles(text, shingleSize):
    return [text[i:i+shingleSize] for i in xrange(0,len(text)-(shingleSize+1))]


def minhash(shinglesList, seed):
    random.seed(seed)
    minhash = -1
    for shingle in shinglesList:
        cwFactory = CWHashingFamily(BINS, PRIME)
        numericHash = cwFactory.numericFunction(random.randrange(1,PRIME), random.randrange(0,PRIME))
        stringHash = cwFactory.stringFunction(random.randrange(0,PRIME),numericHash)
        shHash = stringHash.hash(shingle)
        if ((minhash == -1) or (shHash < minhash)):
            minhash = shHash
    return minhash

In [3]:
ID = 0
TEXT = 9
R = 4
B = 8

trainRDD = sc.textFile("train_sin_repeticiones.csv").map(lambda line: line.split(","))

trainRDD = trainRDD.filter(lambda line: line[ID] != "Id")

trainRDD = trainRDD.map(lambda line: (line[ID], shingles(line[TEXT],SHINGLES_LEN)))

trainRDD = trainRDD.map(lambda line: (line[0], [ minhash(line[1], i) for i in xrange(0, R*B) ] ))

print trainRDD.take(10)


[(u'492346', [7, 0, 2, 0, 4, 1, 21, 4, 6, 2, 0, 1, 2, 2, 0, 0, 3, 8, 0, 0, 6, 6, 1, 0, 0, 2, 0, 4, 2, 0, 0, 0]), (u'343811', [6, 2, 0, 2, 0, 2, 3, 0, 0, 4, 3, 4, 7, 0, 2, 11, 1, 0, 6, 8, 1, 1, 3, 0, 2, 0, 0, 12, 1, 1, 6, 4]), (u'347257', [1, 28, 13, 19, 0, 10, 0, 1, 3, 23, 15, 6, 17, 1, 19, 24, 3, 4, 0, 31, 14, 5, 0, 6, 10, 9, 2, 2, 2, 7, 0, 4]), (u'225529', [48, 87, 96, 49, 67, 13, 11, 11, 14, 15, 43, 42, 26, 55, 0, 203, 59, 17, 17, 7, 73, 10, 38, 41, 43, 99, 20, 77, 75, 93, 69, 14]), (u'485488', [0, 1, 0, 2, 4, 4, 5, 0, 1, 2, 0, 1, 1, 2, 2, 6, 9, 2, 1, 0, 0, 0, 9, 1, 4, 4, 2, 5, 6, 5, 0, 2]), (u'74981', [12, 0, 1, 3, 2, 1, 0, 3, 0, 2, 0, 4, 1, 0, 0, 8, 1, 2, 1, 3, 2, 1, 10, 1, 0, 1, 0, 0, 1, 0, 0, 5]), (u'204180', [1, 10, 0, 0, 2, 0, 6, 2, 2, 4, 0, 1, 1, 1, 1, 0, 7, 3, 10, 1, 3, 3, 1, 3, 0, 0, 2, 0, 0, 1, 0, 1]), (u'454372', [1, 0, 0, 1, 2, 0, 0, 1, 4, 2, 4, 0, 2, 4, 1, 0, 5, 0, 6, 0, 14, 0, 0, 2, 0, 4, 2, 2, 3, 11, 2, 0]), (u'112235', [17, 2, 6, 1, 17, 1, 16, 4, 24, 5, 5, 22, 7, 3, 

In [4]:
from LSH import LSH
from Record import Record

lsh = LSH(R,B)

records = trainRDD.flatMap(lambda line: [lsh.add(Record(line))])


In [5]:
text = "for about 5 years, especially during the winter months, i used e down with several very painful sore throats, many of which would escalate into strep throat or a sinus infection.  i haven't had one since i started drinking kukicha tea 2 years ago.  it's isn't carried in many stores so i'm glad i can get it through amazon now and that it's eligible for free shipping.i'm not a doctor and this isn't medical advice, but if you are having a problem with sore throats this is worth a shot."
test = (459662,shingles(text,4))
test = (test[0], [ minhash(test[1], i) for i in xrange(0, R*B) ] )
testRecord = Record(test)

In [6]:
print lsh.getAllSimilarRecords(testRecord)

IndexError: list index out of range